# Geração da Tabela Base de Origem

In [29]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import streamlit as st

def get_fii_table():
    url = "https://www.fundsexplorer.com.br/ranking"

    options = Options()
    # options.add_argument("--headless=new")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    try:
        driver.get(url)
        time.sleep(2)  # espera inicial

        # --- 0) Tenta fechar/aceitar o cookie banner por vários caminhos ---
        cookie_selectors = [
            'button[data-test="accept"]',
            'button#hs-eu-confirmation-button',             # exemplos comuns
            'button[aria-label*="aceitar"]',
            'button:contains("Aceitar")',                   # fallback textual (pode não funcionar no CSS)
            'div#hs-en-cookie-confirmation-buttons-area button',
            'button:contains("Aceitar todos")'
        ]
        # tentar por texto também (mais confiável)
        texts_to_try = ["Aceitar todos", "Aceitar", "OK", "Fechar"]

        # 1) tentar clique por seletores diretos
        for sel in cookie_selectors:
            try:
                els = driver.find_elements(By.CSS_SELECTOR, sel)
                if els:
                    for e in els:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", e)
                            driver.execute_script("arguments[0].click();", e)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    # se algum botão foi clicado, pausa e tenta prosseguir
                    time.sleep(0.6)
            except Exception:
                pass

        # 2) tentar clicar por texto (procura por botões/links)
        for txt in texts_to_try:
            try:
                candidates = driver.find_elements(By.XPATH, f"//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}') or //a[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}')]]")
                if candidates:
                    for c in candidates:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", c)
                            driver.execute_script("arguments[0].click();", c)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    time.sleep(0.6)
            except Exception:
                pass

        # 3) se ainda existir o elemento de cookie conhecido, remove via JS (forçado)
        try:
            driver.execute_script("""
                var el = document.getElementById('hs-en-cookie-confirmation-buttons-area');
                if (el) { el.remove(); }
                var el2 = document.querySelector('[id^=hs-en-cookie]'); if(el2) el2.remove();
                var overlays = document.querySelectorAll('.cookie, .cookies, .hs-cookie-banner'); 
                overlays.forEach(e => e.remove());
            """)
            time.sleep(0.4)
        except Exception:
            pass

        # --- 4) abrir o menu de colunas (scroll + click via JS para evitar intercept) ---
        botao_colunas = wait.until(EC.presence_of_element_located((By.ID, "colunas-ranking__select-button")))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", botao_colunas)
        try:
            # preferencial: click via JS para evitar intercept
            driver.execute_script("arguments[0].click();", botao_colunas)
        except Exception:
            # fallback: webdriver click
            botao_colunas.click()
        time.sleep(0.6)

        # --- 5) clicar em "todos" (label) ---
        label_todos = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'label[for="colunas-ranking__todos"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", label_todos)
        try:
            driver.execute_script("arguments[0].click();", label_todos)
        except Exception:
            label_todos.click()
        time.sleep(1.0)  # deixa o JS atualizar a tabela

        # --- 6) esperar a tabela estar populada ---
        def tabela_populada(driver):
            try:
                rows = driver.find_elements(By.CSS_SELECTOR, ".default-fiis-table__container__table tbody tr")
                # contar linhas não-vazias
                count = 0
                for r in rows:
                    tds = r.find_elements(By.TAG_NAME, "td")
                    if any(td.text.strip() for td in tds):
                        count += 1
                return count > 5  # ajuste se precisar
            except:
                return False

        wait.until(tabela_populada)

        # --- 7) pegar HTML da tabela ---
        tabela = driver.find_element(By.CSS_SELECTOR, ".default-fiis-table__container__table")
        html = tabela.get_attribute("outerHTML")
        df = pd.read_html(html)[0]
        return df

    finally:
        driver.quit()

#Call the function to test
df_fiis = get_fii_table()

def carregar_dados(df=df_fiis):

    df = df.dropna(subset=[
        'P/VP',
        'DY (3M) Acumulado',
        'DY (6M) Acumulado',
        'DY (12M) Acumulado',
        'Liquidez Diária (R$)',
        'Patrimônio Líquido',
        'Num. Cotistas',
        'Preço Atual (R$)'
    ])

    df['P/VP'] = df['P/VP'] / 100

    for col in ['DY (3M) Acumulado', 'DY (6M) Acumulado', 'DY (12M) Acumulado']:
        df[col] = (
            df[col].astype(str)
            .str.replace('%', '', regex=False)
            .str.replace('.', '', regex=False)
            .str.replace(',', '.', regex=False)
            .astype(float)
        )

    df['Liquidez Diária (R$)'] = (
        df['Liquidez Diária (R$)']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000_000
    )

    df['Patrimônio Líquido'] = (
        df['Patrimônio Líquido']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000_000
    )

    df['Num. Cotistas'] = (
        df['Num. Cotistas']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000
    )

    df['Preço Atual (R$)'] = (
        df['Preço Atual (R$)']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )

    df['Último Dividendo'] = (
        df['Último Dividendo']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )

    df.rename(columns={
        'Liquidez Diária (R$)': 'Liquidez Diária (milhões R$)',
        'Patrimônio Líquido': 'Patrimônio Líquido (milhões R$)',
        'Num. Cotistas': 'Num. Cotistas (milhares)'
    }, inplace=True)

    return df

df_fiis = carregar_dados(df_fiis)
today_str = pd.Timestamp.today().strftime('%Y-%m-%d')
df_fiis.to_parquet(f'df_fiis/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'df_fiis/df_fiis_{today_str}.parquet', index=False)

# Salvando os Top 10 FIIs Descontados com Qualidade
def filtrar_fiis_descontados_com_qualidade(df):
    filtros = (
        (df["P/VP"] >= 0.8) &
        (df["P/VP"] < 1.0) &
        (df["DY (3M) Acumulado"] >= 2.4) &
        (df["DY (6M) Acumulado"] >= 4.8) &
        (df["DY (12M) Acumulado"] >= 9.6) &
        (df["Liquidez Diária (milhões R$)"] >= 1) &
        (df["Patrimônio Líquido (milhões R$)"] >= 500) &
        (df["Num. Cotistas (milhares)"] >= 10)
    )
    return df[filtros].copy()

df_filtrados = filtrar_fiis_descontados_com_qualidade(df_fiis)

df_top10 = (
    df_filtrados
    .sort_values("DY (12M) Acumulado", ascending=False)
    .head(15)
    .sort_values("P/VP")
    .head(10)
)

df_top10.to_parquet(f'hist_top10/hist_top10_{today_str}.parquet', index=False)

C:\Users\danie\AppData\Local\Temp\ipykernel_18964\729001205.py:126: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_18964\729001205.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['P/VP'] = df['P/VP'] / 100
C:\Users\danie\AppData\Local\Temp\ipykernel_18964\729001205.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [30]:
df_fiis = pd.read_parquet("df_fiis.parquet")
df_fiis.head()

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,...,DY Patrimonial,Variação Patrimonial,Rentab. Patr. Período,Rentab. Patr. Acumulada,Quant. Ativos,Volatilidade,Num. Cotistas,Tax. Gestão,Tax. Performance,Tax. Administração
0,AAGR11,Indefinido,12299,"13.345,09",NaN,146,"1,59 %","1,59 %","6,32 %","16,17 %",...,"0,00 %","0,00 %","0,00 %","0,00 %",0,6127,0,None,None,None
1,AAZQ11,Indefinido,849,"413.207,36",97.0,012,"1,47 %","4,51 %","9,22 %","18,95 %",...,"1,39 %","0,00 %","0,00 %","0,00 %",0,1646,30.064,None,"1,20 % a.a","10,00 % a.a"
2,ABCP11,Shoppings,7808,"76.833,91",72.0,065,"0,86 %","2,39 %","4,93 %","9,71 %",...,"0,59 %","20,08 %","20,79 %","29,39 %",1,2619,14.130,None,"0,10 % a.a",None
3,AFHI11,Papéis,9456,"798.950,27",100.0,101,"1,06 %","3,21 %","6,72 %","13,71 %",...,"1,07 %","-0,45 %","0,61 %","6,92 %",14,958,37.566,None,"1,00 % a.a",None
4,AGRX11,Indefinido,802,"264.792,77",75.0,012,"1,49 %","4,49 %","9,22 %","18,06 %",...,"1,13 %","0,00 %","0,00 %","0,00 %",0,2328,18.900,None,"1,00 % a.a","10,00 % a.a"


In [35]:
import urllib.parse
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui

options = Options()
options.add_argument(r"--user-data-dir=C:\selenium\chrome_profile_whatsapp")
options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(options=options)

numero = "5513981832920"
mensagem = "Olá! Tudo bem? Teste FII."

lista_usuarios = {
    "5513981832920" : ['Daniel','CACR11', 'HGLG11'],
    "5527998976226" : ['Esther','XPML11', 'BTAL11']
}
time.sleep(10)
for numero in lista_usuarios.keys():
    mensagem = f'''Opa Boa noite! Tudo bem, {lista_usuarios[numero][0]}? Você pediu para eu validar os seguintes : {', '.join(lista_usuarios[numero][1:])}\n{lista_usuarios[numero][1]} : {df_fiis[df_fiis['Fundos'] == lista_usuarios[numero][1]]['Dividend Yield'].iloc[0]} \nQualquer dúvida estou à disposição!'''
    link = f"https://wa.me/{numero}?text={urllib.parse.quote(mensagem)}"
    print(link)
    # Abre o link do chat
    driver.get(link)

    wait = WebDriverWait(driver, 30)

    # Botão "Continuar para o chat"
    btn_continuar = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/section/div/div/div/div[2]/div[4]/a[2]/span")
        )
    )
    btn_continuar.click()
    time.sleep(10)  # Espera o chat carregar
    # Pressiona ENTER (envia a mensagem)
    pyautogui.press("enter")
    time.sleep(1)
    pyautogui.hotkey('ctrl', 'w')

driver.quit()


https://wa.me/5513981832920?text=Opa%20Boa%20noite%21%20Tudo%20bem%2C%20Daniel%3F%20Voc%C3%AA%20pediu%20para%20eu%20validar%20os%20seguintes%20%3A%20CACR11%2C%20HGLG11%0ACACR11%20%3A%201%2C74%20%25%20%0AQualquer%20d%C3%BAvida%20estou%20%C3%A0%20disposi%C3%A7%C3%A3o%21
https://wa.me/5527998976226?text=Opa%20Boa%20noite%21%20Tudo%20bem%2C%20Esther%3F%20Voc%C3%AA%20pediu%20para%20eu%20validar%20os%20seguintes%20%3A%20XPML11%2C%20BTAL11%0AXPML11%20%3A%200%2C86%20%25%20%0AQualquer%20d%C3%BAvida%20estou%20%C3%A0%20disposi%C3%A7%C3%A3o%21
